## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam_0.000015'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.75 ## EPSILON
lr_rl = 0.009 ## LEARNING RATE RL
disc_fac = 0.999 ## DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = model_name + str(lr_rl)##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (6849, 15)
ds_val.shape (1476, 15)
ds_test.shape (1479, 15)
ds_rl.shape (1845, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 6849, Val 1476


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (6849, 15)
shape ds_val  : (1476, 15)
shape ds_test : (1479, 15)
shape ds_rl : (1845, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1476, 5)
shape x_train_query: (6849, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1476,)
shape x_train_tree: (6849,)
----------y_data------------
shape y_val  : (1476,)
shape y_train: (6849,)

----------TEST DATA----------
shape x_test_tree : (1479,)
shape x_test_query: (1479, 5)
shape y_test      : (1479,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (1845,)
shape x_rl_query: (1845, 5)
shape y_rl      : (1845,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1282, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.112900
1         1.983621
2         1.503600
3         1.404750
4         1.669234
           ...    
1474    102.446694
1475    148.000000
1476    107.995152
1477    113.000000
1478    102.963054
Name: time, Length: 1479, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [3.377333641052246,
  2.169917583465576,
  25.933746337890625,
  97.41815185546875,
  24.397424697875977,
  17.583248138427734,
  14.958420753479004,
  3.0458641052246094,
  18.781415939331055,
  18.18781280517578,
  23.263587951660156,
  2.7690536975860596,
  20.075891494750977,
  19.975412368774414,
  17.495756149291992,
  3.177464485168457,
  17.256366729736328,
  3.2106170654296875,
  3.8193187713623047,
  25.20635414123535,
  17.426471710205078,
  18.395904541015625,
  26.87457275390625,
  4.034188270568848,
  19.281034469604492,
  18.82175636291504,
  67.76089477539062,
  3.2232518196105957,
  18.815237045288086,
  58.298465728759766,
  3.9630775451660156,
  63.50189971923828,
  4.728207588195801,
  18.954923629760742,
  2.6525304317474365,
  3.5804715156555176,
  7.479750633239746,
  3.2000560760498047,
  17.003477096557617,
  39.84954071044922,
  3.0051674842834473,
  102.66240692138672,
  13.741914749145508,
  18.061138153076172,
  18.9135799407959,
  19.6443862915039

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (1845, 67)
shape x_rl_tree : (1845,)
shape x_rl_query: (1845, 5)
shape y_rl      : (1845,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
no_history = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 1845


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [23]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2022-01-01 02:10:36.400015
2022-01-01 02:11:16.254524 : 10/1845, index 9 evaluating
2022-01-01 02:12:03.480412 : 20/1845, index 19 evaluating
2022-01-01 02:12:56.220578 : 30/1845, index 29 evaluating
2022-01-01 02:13:42.253187 : 40/1845, index 39 evaluating
2022-01-01 02:14:32.345161 : 50/1845, index 49 evaluating
2022-01-01 02:15:22.620527 : 60/1845, index 59 evaluating
2022-01-01 02:16:15.753499 : 70/1845, index 69 evaluating
2022-01-01 02:17:07.357294 : 80/1845, index 79 evaluating
2022-01-01 02:18:03.218345 : 90/1845, index 89 evaluating
2022-01-01 02:18:56.257783 : 100/1845, index 99 evaluating
2022-01-01 02:20:06.027966 : 110/1845, index 109 evaluating
2022-01-01 02:20:55.394067 : 120/1845, index 119 evaluating
2022-01-01 02:21:41.628646 : 130/1845, index 129 evaluating
2022-01-01 02:22:31.963263 : 140/1845, index 139 evaluating
2022-01-01 02:23:22.459768 : 150/1845, index 149 evaluating
2022-01-01 02:24:15.786131 : 160/1845, index 159 evaluating
2022-01-01 02:25:06.467559

2022-01-01 04:15:01.227833 : 1370/1845, index 1369 evaluating
2022-01-01 04:16:01.948575 : 1380/1845, index 1379 evaluating
2022-01-01 04:16:57.753314 : 1390/1845, index 1389 evaluating
2022-01-01 04:17:53.861723 : 1400/1845, index 1399 evaluating
2022-01-01 04:18:46.841559 : 1410/1845, index 1409 evaluating
2022-01-01 04:19:39.699512 : 1420/1845, index 1419 evaluating
2022-01-01 04:20:36.185350 : 1430/1845, index 1429 evaluating
2022-01-01 04:21:30.780216 : 1440/1845, index 1439 evaluating
2022-01-01 04:22:26.748318 : 1450/1845, index 1449 evaluating
2022-01-01 04:23:24.280537 : 1460/1845, index 1459 evaluating
2022-01-01 04:24:22.988039 : 1470/1845, index 1469 evaluating
2022-01-01 04:25:20.440898 : 1480/1845, index 1479 evaluating
2022-01-01 04:26:13.720660 : 1490/1845, index 1489 evaluating
2022-01-01 04:27:17.775630 : 1500/1845, index 1499 evaluating
2022-01-01 04:28:17.794128 : 1510/1845, index 1509 evaluating
2022-01-01 04:29:09.461268 : 1520/1845, index 1519 evaluating
2022-01-

In [24]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_no_history = ql_df[no_history]
ql_df_no_history.to_csv(URL + 'rl_csvs/ql_df_no_history'+str(n)+'.csv',index=False)

In [25]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [26]:
ql_df_no_history

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,51.191635,53.554314,50.0,50.0,1.419994,12.633146,1.191635,3.554314,1.191635,3.554314,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,0,1,51.493649,51.326431,50.0,50.0,2.230986,1.759420,1.493649,1.326431,1.493649,1.326431,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
2,0,2,50.661934,52.939232,50.0,50.0,0.438156,8.639084,0.661934,2.939232,0.661934,2.939232,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
3,0,3,53.087234,51.285042,50.0,50.0,9.531017,1.651332,3.087234,1.285042,3.087234,1.285042,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
4,0,4,52.310478,51.615990,50.0,50.0,5.338310,2.611423,2.310478,1.615990,2.310478,1.615990,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368995,1844,195,85.270035,77.919075,119.0,119.0,1137.710553,1687.642398,33.729965,41.080925,33.729965,41.080925,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368996,1844,196,93.040985,78.028358,119.0,119.0,673.870454,1678.675411,25.959015,40.971642,25.959015,40.971642,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368997,1844,197,84.372078,78.434311,119.0,119.0,1199.092986,1645.575131,34.627922,40.565689,34.627922,40.565689,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368998,1844,198,82.009796,76.185852,119.0,119.0,1368.275181,1833.051265,36.990204,42.814148,36.990204,42.814148,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."


In [27]:
ql_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,51.046520,50.489388,50.0,50.0,1.095205,0.239500,1.046520,0.489388,1.046520,0.489388,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,1,199,44.581898,45.094437,50.0,50.0,29.355832,24.064552,5.418102,4.905563,5.418102,4.905563,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
2,2,199,47.070812,46.178513,50.0,50.0,8.580141,14.603766,2.929188,3.821487,2.929188,3.821487,"['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V..."
3,3,199,60.280071,58.971256,50.0,50.0,105.679865,80.483439,10.280071,8.971256,10.280071,8.971256,"['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire..."
4,4,199,46.642292,49.069252,50.0,50.0,11.274203,0.866292,3.357708,0.930748,3.357708,0.930748,"['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1840,199,81.858772,76.171761,109.0,109.0,736.646242,1077.693305,27.141228,32.828239,27.141228,32.828239,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1841,1841,199,57.992466,57.575935,130.0,130.0,5185.084957,5245.245138,72.007534,72.424065,72.007534,72.424065,"['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']"
1842,1842,199,84.246460,81.754021,119.0,119.0,1207.808545,1387.262975,34.753540,37.245979,34.753540,37.245979,"['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V..."
1843,1843,199,85.973198,82.075058,118.0,118.0,1025.716050,1290.601459,32.026802,35.924942,32.026802,35.924942,"['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA..."


## Sarsa Multival

###### Parameters

In [34]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [35]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

Start: 2022-01-01 07:01:05.109326
2022-01-01 07:01:47.545199 : 10/1845, index 9 evaluating
2022-01-01 07:02:35.912155 : 20/1845, index 19 evaluating
2022-01-01 07:03:30.458423 : 30/1845, index 29 evaluating
2022-01-01 07:04:18.180952 : 40/1845, index 39 evaluating
2022-01-01 07:05:09.265159 : 50/1845, index 49 evaluating
2022-01-01 07:06:01.129947 : 60/1845, index 59 evaluating
2022-01-01 07:06:54.737240 : 70/1845, index 69 evaluating
2022-01-01 07:07:47.976113 : 80/1845, index 79 evaluating
2022-01-01 07:08:45.416234 : 90/1845, index 89 evaluating
2022-01-01 07:09:39.968935 : 100/1845, index 99 evaluating
2022-01-01 07:10:51.055726 : 110/1845, index 109 evaluating
2022-01-01 07:11:41.793025 : 120/1845, index 119 evaluating
2022-01-01 07:12:29.459128 : 130/1845, index 129 evaluating
2022-01-01 07:13:21.437309 : 140/1845, index 139 evaluating
2022-01-01 07:14:13.551978 : 150/1845, index 149 evaluating
2022-01-01 07:15:08.529780 : 160/1845, index 159 evaluating
2022-01-01 07:16:00.959505

2022-01-01 09:07:48.003541 : 1370/1845, index 1369 evaluating
2022-01-01 09:08:48.575244 : 1380/1845, index 1379 evaluating
2022-01-01 09:09:45.421283 : 1390/1845, index 1389 evaluating
2022-01-01 09:10:41.763988 : 1400/1845, index 1399 evaluating
2022-01-01 09:11:35.559359 : 1410/1845, index 1409 evaluating
2022-01-01 09:12:29.433171 : 1420/1845, index 1419 evaluating
2022-01-01 09:13:26.487449 : 1430/1845, index 1429 evaluating
2022-01-01 09:14:21.887027 : 1440/1845, index 1439 evaluating
2022-01-01 09:15:18.404336 : 1450/1845, index 1449 evaluating
2022-01-01 09:16:16.349345 : 1460/1845, index 1459 evaluating
2022-01-01 09:17:15.722707 : 1470/1845, index 1469 evaluating
2022-01-01 09:18:13.706204 : 1480/1845, index 1479 evaluating
2022-01-01 09:19:07.076493 : 1490/1845, index 1489 evaluating
2022-01-01 09:20:11.872489 : 1500/1845, index 1499 evaluating
2022-01-01 09:21:12.710557 : 1510/1845, index 1509 evaluating
2022-01-01 09:22:05.042155 : 1520/1845, index 1519 evaluating
2022-01-

In [36]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_no_history = sarsa_df[no_history]
sarsa_df_no_history.to_csv(URL + 'rl_csvs/sarsa_df_no_history'+str(n)+'.csv',index=False)

In [37]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [38]:
sarsa_df_no_history

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,53.103226,51.257038,50.0,50.0,9.630010,1.580145,3.103226,1.257038,3.103226,1.257038,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,0,1,51.920059,50.648239,50.0,50.0,3.686627,0.420214,1.920059,0.648239,1.920059,0.648239,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
2,0,2,50.956821,52.000187,50.0,50.0,0.915507,4.000748,0.956821,2.000187,0.956821,2.000187,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
3,0,3,51.672070,50.999065,50.0,50.0,2.795817,0.998132,1.672070,0.999065,1.672070,0.999065,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
4,0,4,50.877392,51.445934,50.0,50.0,0.769816,2.090726,0.877392,1.445934,0.877392,1.445934,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368995,1844,195,85.338722,81.351837,119.0,119.0,1133.081621,1417.384165,33.661278,37.648163,33.661278,37.648163,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V..."
368996,1844,196,89.545998,89.401985,119.0,119.0,867.538256,876.042482,29.454002,29.598015,29.454002,29.598015,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V..."
368997,1844,197,85.181023,85.119278,119.0,119.0,1143.723229,1147.903326,33.818977,33.880722,33.818977,33.880722,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V..."
368998,1844,198,90.275040,88.561920,119.0,119.0,825.123346,926.476704,28.724960,30.438080,28.724960,30.438080,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V..."


In [39]:
sarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,51.962555,52.502426,50.0,50.0,3.851622,6.262137,1.962555,2.502426,1.962555,2.502426,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,1,199,45.129246,42.771748,50.0,50.0,23.724247,52.247633,4.870754,7.228252,4.870754,7.228252,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
2,2,199,50.075420,46.337460,50.0,50.0,0.005688,13.414202,0.075420,3.662540,0.075420,3.662540,"['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V..."
3,3,199,60.702885,56.751373,50.0,50.0,114.551740,45.581041,10.702885,6.751373,10.702885,6.751373,"['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire..."
4,4,199,50.901825,47.723610,50.0,50.0,0.813288,5.181952,0.901825,2.276390,0.901825,2.276390,"['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1840,199,84.039284,76.283821,109.0,109.0,623.037356,1070.348361,24.960716,32.716179,24.960716,32.716179,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1841,1841,199,58.215675,60.481613,130.0,130.0,5152.989265,4832.806109,71.784325,69.518387,71.784325,69.518387,"['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']"
1842,1842,199,78.049393,80.219879,119.0,119.0,1676.952238,1503.897773,40.950607,38.780121,40.950607,38.780121,"['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V..."
1843,1843,199,87.577271,84.448601,118.0,118.0,925.542470,1125.696390,30.422729,33.551399,30.422729,33.551399,"['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA..."


## Expected-Sarsa Multival

###### Parameters

In [40]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = 0.9
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [41]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

Start: 2022-01-01 09:54:12.892972
2022-01-01 09:54:55.988721 : 10/1845, index 9 evaluating
2022-01-01 09:55:45.421753 : 20/1845, index 19 evaluating
2022-01-01 09:56:40.048355 : 30/1845, index 29 evaluating
2022-01-01 09:57:27.905538 : 40/1845, index 39 evaluating
2022-01-01 09:58:20.071232 : 50/1845, index 49 evaluating
2022-01-01 09:59:12.242046 : 60/1845, index 59 evaluating
2022-01-01 10:00:07.667327 : 70/1845, index 69 evaluating
2022-01-01 10:01:00.939556 : 80/1845, index 79 evaluating
2022-01-01 10:01:58.579694 : 90/1845, index 89 evaluating
2022-01-01 10:02:52.810248 : 100/1845, index 99 evaluating
2022-01-01 10:04:04.797339 : 110/1845, index 109 evaluating
2022-01-01 10:04:55.513921 : 120/1845, index 119 evaluating
2022-01-01 10:05:43.697102 : 130/1845, index 129 evaluating
2022-01-01 10:06:35.814869 : 140/1845, index 139 evaluating
2022-01-01 10:07:27.837937 : 150/1845, index 149 evaluating
2022-01-01 10:08:22.945454 : 160/1845, index 159 evaluating
2022-01-01 10:09:15.171928

2022-01-01 12:01:43.474913 : 1370/1845, index 1369 evaluating
2022-01-01 12:02:45.088892 : 1380/1845, index 1379 evaluating
2022-01-01 12:03:41.818678 : 1390/1845, index 1389 evaluating
2022-01-01 12:04:38.397720 : 1400/1845, index 1399 evaluating
2022-01-01 12:05:32.261296 : 1410/1845, index 1409 evaluating
2022-01-01 12:06:26.236342 : 1420/1845, index 1419 evaluating
2022-01-01 12:07:23.514893 : 1430/1845, index 1429 evaluating
2022-01-01 12:08:18.934764 : 1440/1845, index 1439 evaluating
2022-01-01 12:09:16.387443 : 1450/1845, index 1449 evaluating
2022-01-01 12:10:14.800550 : 1460/1845, index 1459 evaluating
2022-01-01 12:11:15.002297 : 1470/1845, index 1469 evaluating
2022-01-01 12:12:13.240023 : 1480/1845, index 1479 evaluating
2022-01-01 12:13:06.981047 : 1490/1845, index 1489 evaluating
2022-01-01 12:14:12.199211 : 1500/1845, index 1499 evaluating
2022-01-01 12:15:13.291177 : 1510/1845, index 1509 evaluating
2022-01-01 12:16:05.610112 : 1520/1845, index 1519 evaluating
2022-01-

In [42]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_no_history = exsarsa_df[no_history]
exsarsa_df_no_history.to_csv(URL + 'rl_csvs/exsarsa_df_no_history'+str(n)+'.csv',index=False)

In [43]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [44]:
exsarsa_df_no_history

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,52.636162,50.967003,50.0,50.0,6.949349,0.935095,2.636162,0.967003,2.636162,0.967003,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,0,1,49.955761,51.308308,50.0,50.0,0.001957,1.711669,0.044239,1.308308,0.044239,1.308308,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
2,0,2,52.177555,53.057480,50.0,50.0,4.741746,9.348183,2.177555,3.057480,2.177555,3.057480,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
3,0,3,52.226986,50.702042,50.0,50.0,4.959466,0.492862,2.226986,0.702042,2.226986,0.702042,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
4,0,4,50.697334,51.537434,50.0,50.0,0.486275,2.363702,0.697334,1.537434,0.697334,1.537434,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368995,1844,195,84.997856,75.066383,119.0,119.0,1156.145787,1930.162671,34.002144,43.933617,34.002144,43.933617,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368996,1844,196,87.428635,75.676979,119.0,119.0,996.751110,1876.884143,31.571365,43.323021,31.571365,43.323021,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368997,1844,197,86.263908,71.272911,119.0,119.0,1071.651694,2277.875018,32.736092,47.727089,32.736092,47.727089,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368998,1844,198,82.671547,80.073235,119.0,119.0,1319.756502,1515.293068,36.328453,38.926765,36.328453,38.926765,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."


In [45]:
exsarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,50.514805,51.952644,50.0,50.0,0.265024,3.812820,0.514805,1.952644,0.514805,1.952644,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,1,199,44.696262,42.911312,50.0,50.0,28.129633,50.249496,5.303738,7.088688,5.303738,7.088688,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
2,2,199,50.123383,51.810818,50.0,50.0,0.015223,3.279061,0.123383,1.810818,0.123383,1.810818,"['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V..."
3,3,199,59.040184,61.645340,50.0,50.0,81.724927,135.613943,9.040184,11.645340,9.040184,11.645340,"['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire..."
4,4,199,48.425652,44.434872,50.0,50.0,2.478573,30.970653,1.574348,5.565128,1.574348,5.565128,"['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1840,199,84.952347,72.844208,109.0,109.0,578.289624,1307.241312,24.047653,36.155792,24.047653,36.155792,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1841,1841,199,57.980598,58.477211,130.0,130.0,5186.794200,5115.509347,72.019402,71.522789,72.019402,71.522789,"['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']"
1842,1842,199,80.655891,74.678368,119.0,119.0,1470.270663,1964.407097,38.344109,44.321632,38.344109,44.321632,"['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_VAR_V..."
1843,1843,199,84.685951,88.906242,118.0,118.0,1109.825845,846.446733,33.314049,29.093758,33.314049,29.093758,"['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA..."


In [46]:
1+1

2

### Archivos enteros

In [47]:
import os
path = "/home/school/math/final_analysis.csv"

In [48]:
os.remove(URL + 'rl_csvs/ql_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/ccarmona/memoria/dataset/rl_csvs/ql_df.csv'

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)